#  Government Services Assistant - AI Chatbot

This notebook demonstrates how to use an AI-powered assistant to answer questions about  government services and procedures, using the Firecrawl API and a friendly, step-by-step conversational approach.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DhivyaBharathy-web/PraisonAI/blob/main/examples/cookbooks/Chile_Government_Services_Assistant.ipynb)


# Install dependencies

In [ ]:
!pip install flask firecrawl praisonaiagents google-genai python-dotenv deep-translator

# Set API Keys

In [ ]:
import os

os.environ['FIRECRAWL_API_KEY'] = "your api key here"
os.environ['OPENAI_API_KEY'] = "your api key here"

# Import Libraries & Translator

In [ ]:
from firecrawl import FirecrawlApp, ScrapeOptions
from deep_translator import GoogleTranslator
import re

def translate_to_spanish(text):
    try:
        return GoogleTranslator(source='auto', target='es').translate(text)
    except Exception as e:
        print("Translation to Spanish failed:", e)
        return text

def translate_to_english(text):
    try:
        # Remove Markdown images and None values before translation
        text = str(text).replace("None", "")
        text = re.sub(r'!\[.*?\]\(.*?\)', '', text)
        return GoogleTranslator(source='auto', target='en').translate(text)
    except Exception as e:
        print("Translation to English failed:", e)
        return text

# Firecrawl Tool Class

In [ ]:
class FirecrawlTool:
    def __init__(self, api_key, instruction: str, template: str):
        if not api_key:
            raise ValueError("Firecrawl API key not provided.")
        self.app = FirecrawlApp(api_key=api_key)
        self.instruction = instruction
        self.template = template

    def search(self, search: str) -> str:
        if not search or len(search) < 5:
            return "Error: Please provide a valid search query (at least 5 characters)."
        response_md = ""
        try:
            search_result = self.app.search(
                query=self.instruction + search,
                limit=2,
                country="cl",
                lang="es",  # Always search in Spanish for best results
                scrape_options=ScrapeOptions(formats=["markdown", "links"])
            )
            if search_result and hasattr(search_result, 'data') and search_result.data:
                filtered_results = [
                    result for result in search_result.data
                    if str(result.get("url", "")).startswith("https://www.chileatiende.gob.cl/fichas") and not str(result.get("url", "")).endswith("pdf")
                ]
                if filtered_results:
                    for num, result in enumerate(filtered_results, start=1):
                        response_md += self.template.format(
                            result_number=num,
                            page_title=str(result.get("title", "")),
                            page_url=str(result.get("url", "")),
                            page_content=str(result.get("markdown", ""))
                        )
                    return response_md
                else:
                    return None
            else:
                return None
        except Exception as e:
            return f"Error during search: {e}"

# Firecrawl Prompt Template

In [ ]:
FIRECRAWL_INSTRUCTION = "ChileAtiende: "
FIRECRAWL_TEMPLATE = """
# Result {result_number}

## Page Name:
"{page_title}"

## URL:
{page_url}

## Content:
{page_content}

"""

# Initialize Firecrawl Tool

In [ ]:
firecrawl_tool = FirecrawlTool(
    api_key=os.environ['FIRECRAWL_API_KEY'],
    instruction=FIRECRAWL_INSTRUCTION,
    template=FIRECRAWL_TEMPLATE
)

# Main Chat Loop

In [ ]:
print("Hello! I am your ChileAtiende assistant, Tomás. How can I help you today?")
print("You can ask me, for example: How to renew your ID card, How to apply for the Winter Bonus, etc.")

while True:
    user_input = input("\nYou: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Tomás: It was a pleasure to help you. Goodbye!")
        break

    # Translate English input to Spanish for Firecrawl
    spanish_query = translate_to_spanish(user_input)
    spanish_answer = firecrawl_tool.search(spanish_query)

    # Only translate if we got a real answer
    if spanish_answer and isinstance(spanish_answer, str) and spanish_answer.strip() and "Error" not in spanish_answer:
        try:
            english_answer = translate_to_english(spanish_answer)
            print("\nTomás (in English):\n", english_answer)
        except Exception as e:
            print(f"\nTomás: I found information, but couldn't translate it. Here it is in Spanish:\n{spanish_answer}\n(Translation error: {e})")
    else:
        print("\nTomás: Sorry, I couldn't find relevant information. Try rephrasing your question or ask about another service.")

Hello! I am your ChileAtiende assistant, Tomás. How can I help you today?
You can ask me, for example: How to renew your ID card, How to apply for the Winter Bonus, etc.

You: exit
Tomás: It was a pleasure to help you. Goodbye!
